In [596]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from scipy import stats
import string

In [559]:
#df = pd.read_csv('NLP_project_kickoff_data.csv', encoding='utf-8')
df = pd.read_csv('NLP_project_kickoff_data.csv', encoding='utf-8')

In [560]:
df.head()

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,NaN,2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ..."
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa..."
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipâ„¢\n""Lip I...",British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',..."
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"Facial Cosmetics, ENT and Thyroid Clinic by Mr...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screenshot', 'dog',..."
4,4,dr_danmarsh,Tummytuck at 6 months post op. Final results t...,Dr Dan Marsh PhD FRCS (plast),"ðŸŒ� London, UK\nðŸ�¥ Co-founder @theplasticsu...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['Brown', 'Black', 'Pink', 'undergarment', 'cl..."


In [561]:
df_emo= pd.read_csv('emojis.csv')

In [562]:
df_emo[df_emo['emoji']=='ðŸŒ']

,emoji,meaning
546,ðŸŒ,J_globe2


In [563]:
for index, i in enumerate(df['biography']):
  i=str(i).replace("�", "")
  for e, m in zip(df_emo.emoji, df_emo.meaning):
     if e in i:
       i=i.replace(e, m+' ')
       df['biography']=i
  print(i)
  
#df.head()


nan
Facial Cosmetics, ENT and Thyroid Clinic by Mr. Mrinal Supriya, an experienced Surgeon. For a Consultation call British Face Clinic on 01604 556244.
Facial Cosmetics, ENT and Thyroid Clinic by Mr. Mrinal Supriya, an experienced Surgeon. For a Consultation call British Face Clinic on 01604 556244.
Facial Cosmetics, ENT and Thyroid Clinic by Mr. Mrinal Supriya, an experienced Surgeon. For a Consultation call British Face Clinic on 01604 556244.
J_globe4  London, UK
N_building6  Co-founder @theplasticsurgerygroup  
B_man â€F_brief GMC Specialist Register Plastic Surgery
L_office8 BAAPS Council & BAPRAS member
J_globe4  London, UK
N_building6  Co-founder @theplasticsurgerygroup  
B_man â€F_brief GMC Specialist Register Plastic Surgery
L_office8 BAAPS Council & BAPRAS member
J_globe4  London, UK
N_building6  Co-founder @theplasticsurgerygroup  
B_man â€F_brief GMC Specialist Register Plastic Surgery
L_office8 BAAPS Council & BAPRAS member
J_globe4  London, UK
N_building6  Co-founder 

In [564]:
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens
0,0,dr_hazem_alfie,"I am Dr Hazem Alfie, Plastic and Cosmetic Surg...",NaN,"J_globe4 London, UK\nN_building6 Co-founder ...",2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['White', 'human', 'face', 'person', 'smile', ..."
1,1,british_face_clinic,British Face Clinic is offering free facial co...,British Face Clinic,"J_globe4 London, UK\nN_building6 Co-founder ...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['White', 'Grey', 'text', 'human', 'face', 'fa..."
2,2,british_face_clinic,"Need fuller lips? Consider PermaLipâ„¢\n""Lip I...",British Face Clinic,"J_globe4 London, UK\nN_building6 Co-founder ...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['White', 'skin', 'lip', 'closeup', 'eyelash',..."
3,3,british_face_clinic,A recent successful Rhinoplasty procedure resu...,British Face Clinic,"J_globe4 London, UK\nN_building6 Co-founder ...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['White', 'Grey', 'text', 'screenshot', 'dog',..."
4,4,dr_danmarsh,Tummytuck at 6 months post op. Final results t...,Dr Dan Marsh PhD FRCS (plast),"J_globe4 London, UK\nN_building6 Co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['Brown', 'Black', 'Pink', 'undergarment', 'cl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,Vincenzo Cassaro,"J_globe4 London, UK\nN_building6 Co-founder ...",8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['Grey', 'Black', 'outdoor', 'aerial', 'photog..."
515,1,viciocassaro,ðŸ‘”,Vincenzo Cassaro,"J_globe4 London, UK\nN_building6 Co-founder ...",2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/CJ03kIkj8cH/,0,0,NaN,2021-11-06T17:39:59.077Z,0.008394,0.011220,0.000874,0,0,0,"['Black', 'person', 'christmas', 'tree', 'clot..."
516,2,viciocassaro,#OneStepCloser,Vincenzo Cassaro,"J_globe4 London, UK\nN_building6 Co-founder ...",1.030000e+18,407,48,36,431,0,1436953461,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/5JvdTIJmWU/,0,0,NaN,2021-11-06T17:39:59.077Z,0.000879,0.001615,0.000563,0,0,0,"['White', 'human', 'face', 'person', 'clothing..."
517,0,viciocassaro,wow.,Vincenzo Cassaro,"J_globe4 London, UK\nN_building6 Co-founder ...",8.710000e+17,407,27,36,437,0,1418073537,j_kvo3dl7njv41lgn55.c75281bab6c50a2a8905d2be3a...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:40:39.623Z,0.001229,0.002201,0.000616,0,0,0,"['Grey', 'Black', 'outdoor', 'aerial', 'photog..."


##Text Lower

In [565]:
for col in ['caption', 'profile_name','biography','tokens']:
  df[col]=df[col].str.lower()
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens
0,0,dr_hazem_alfie,"i am dr hazem alfie, plastic and cosmetic surg...",NaN,"j_globe4 london, uk\nn_building6 co-founder ...",2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['white', 'human', 'face', 'person', 'smile', ..."
1,1,british_face_clinic,british face clinic is offering free facial co...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['white', 'grey', 'text', 'human', 'face', 'fa..."
2,2,british_face_clinic,"need fuller lips? consider permalipâ„¢\n""lip i...",british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['white', 'skin', 'lip', 'closeup', 'eyelash',..."
3,3,british_face_clinic,a recent successful rhinoplasty procedure resu...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['white', 'grey', 'text', 'screenshot', 'dog',..."
4,4,dr_danmarsh,tummytuck at 6 months post op. final results t...,dr dan marsh phd frcs (plast),"j_globe4 london, uk\nn_building6 co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['brown', 'black', 'pink', 'undergarment', 'cl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['grey', 'black', 'outdoor', 'aerial', 'photog..."
515,1,viciocassaro,ðÿ‘”,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/CJ03kIkj8cH/,0,0,NaN,2021-11-06T17:39:59.077Z,0.008394,0.011220,0.000874,0,0,0,"['black', 'person', 'christmas', 'tree', 'clot..."
516,2,viciocassaro,#onestepcloser,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",1.030000e+18,407,48,36,431,0,1436953461,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/5JvdTIJmWU/,0,0,NaN,2021-11-06T17:39:59.077Z,0.000879,0.001615,0.000563,0,0,0,"['white', 'human', 'face', 'person', 'clothing..."
517,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,437,0,1418073537,j_kvo3dl7njv41lgn55.c75281bab6c50a2a8905d2be3a...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:40:39.623Z,0.001229,0.002201,0.000616,0,0,0,"['grey', 'black', 'outdoor', 'aerial', 'photog..."


##Remove punctuation

In [566]:
df['caption'][0]

'i am dr hazem alfie, plastic and cosmetic surgeon based in the ukâ€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦â€¦ #breastaugmentation #breastenlargemnet #vaserliposuction #vaser #rhinoplasty #abdominoplasty #tummytuck #breastreduction'

In [567]:
from functools import reduce

def hashtag_extract(text):
    return re.findall(r"#(\w+)", text)

#Feature Engineering

###Engagement factor 

In [568]:
df['engagement_factor'] =round(df['likes']/df['followers'],2)

In [569]:
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,engagement_factor
0,0,dr_hazem_alfie,"i am dr hazem alfie, plastic and cosmetic surg...",NaN,"j_globe4 london, uk\nn_building6 co-founder ...",2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['white', 'human', 'face', 'person', 'smile', ...",0.54
1,1,british_face_clinic,british face clinic is offering free facial co...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['white', 'grey', 'text', 'human', 'face', 'fa...",0.18
2,2,british_face_clinic,"need fuller lips? consider permalipâ„¢\n""lip i...",british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['white', 'skin', 'lip', 'closeup', 'eyelash',...",0.16
3,3,british_face_clinic,a recent successful rhinoplasty procedure resu...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['white', 'grey', 'text', 'screenshot', 'dog',...",0.12
4,4,dr_danmarsh,tummytuck at 6 months post op. final results t...,dr dan marsh phd frcs (plast),"j_globe4 london, uk\nn_building6 co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['brown', 'black', 'pink', 'undergarment', 'cl...",0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['grey', 'black', 'outdoor', 'aerial', 'photog...",0.06
515,1,viciocassaro,ðÿ‘”,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/CJ03kIkj8cH/,0,0,NaN,2021-11-06T17:39:59.077Z,0.008394,0.011220,0.000874,0,0,0,"['black', 'person', 'christmas', 'tree', 'clot...",0.20
516,2,viciocassaro,#onestepcloser,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",1.030000e+18,407,48,36,431,0,1436953461,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/5JvdTIJmWU/,0,0,NaN,2021-11-06T17:39:59.077Z,0.000879,0.001615,0.000563,0,0,0,"['white', 'human', 'face', 'person', 'clothing...",0.11
517,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,437,0,1418073537,j_kvo3dl7njv41lgn55.c75281bab6c50a2a8905d2be3a...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:40:39.623Z,0.001229,0.002201,0.000616,0,0,0,"['grey', 'black', 'outdoor', 'aerial', 'photog...",0.06


1. discard the user by post count < n
2. Recent k=5 last six post based on date and time stamp  add all 6 likes  /6
3.  most recent timestamp
4. timestamp - which time we get the most likes 
5. feature from caption -- no of words 

In [570]:
# df['time'] = pd.to_datetime(df['timestamp'])

###deleting the user by pose count < 100

In [571]:
df.shape

(519, 26)

In [598]:
df['Dow'] = df['timstamp_1'].dt.day_name()
df['HoW']= df['timstamp_1'].dt.hour

In [594]:
df['account'].value_counts()

cheshirelashcompany    104
viciocassaro            25
spacex                  25
rozinaalimd             12
abiolarenee             11
                      ... 
drdlanger                1
ionkavounilondon         1
lmskincare               1
harpalclinic             1
dr_hazem_alfie           1
Name: account, Length: 108, dtype: int64

In [595]:
df

,Unnamed: 0,account,caption,profile_name,biography,id,following,likes,posts_count,followers,is_verified,datetime,image_url,url,comments,video_view_count,video_url,timestamp,adultScore,racyScore,goreScore,isBWImg,clipArtType,lineDrawingType,tokens,engagement_factor,timstamp_1,Day of Week,Hour of Week
0,0,dr_hazem_alfie,"i am dr hazem alfie, plastic and cosmetic surg...",NaN,"j_globe4 london, uk\nn_building6 co-founder ...",2.680000e+18,2,15,1,28,0,1634065389,j_kvihwg0g1c283e3byk.0f2035486e2f1b57c59707ec9...,https://www.instagram.com/p/CU8IoDEr-x3/,4,0,NaN,2021-11-02T19:40:32.981Z,0.001138,0.002034,0.000703,0,0,0,"['white', 'human', 'face', 'person', 'smile', ...",0.54,2021-11-02 19:40:32.981000+00:00,Tuesday,19
1,1,british_face_clinic,british face clinic is offering free facial co...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,10,4,56,0,1542635007,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqXTArshaKO/,0,0,NaN,2021-11-02T19:40:36.300Z,0.014004,0.016998,0.002984,0,0,0,"['white', 'grey', 'text', 'human', 'face', 'fa...",0.18,2021-11-02 19:40:36.300000+00:00,Tuesday,19
2,2,british_face_clinic,"need fuller lips? consider permalipâ„¢\n""lip i...",british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,9,4,56,0,1543377164,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqtakFwhYpo/,0,0,NaN,2021-11-02T19:40:36.300Z,0.002261,0.011085,0.011169,0,0,0,"['white', 'skin', 'lip', 'closeup', 'eyelash',...",0.16,2021-11-02 19:40:36.300000+00:00,Tuesday,19
3,3,british_face_clinic,a recent successful rhinoplasty procedure resu...,british face clinic,"j_globe4 london, uk\nn_building6 co-founder ...",1.920000e+18,106,7,4,56,0,1542967109,j_kvihwg0g1c283e3byk.bc7f91a5dec2439a829fcb6e5...,https://www.instagram.com/p/BqhMccDBf5z/,1,0,NaN,2021-11-02T19:40:36.300Z,0.005521,0.009230,0.000926,0,0,0,"['white', 'grey', 'text', 'screenshot', 'dog',...",0.12,2021-11-02 19:40:36.300000+00:00,Tuesday,19
4,4,dr_danmarsh,tummytuck at 6 months post op. final results t...,dr dan marsh phd frcs (plast),"j_globe4 london, uk\nn_building6 co-founder ...",2.400000e+18,109,2778,309,72047,1,1600373084,j_kvihwg0g1c283e3byk.d165e082aac6a78850c61cecd...,https://www.instagram.com/p/CFQBpv2n2sG/,161,0,NaN,2021-11-02T19:40:32.890Z,0.144270,0.989714,0.001252,0,0,0,"['brown', 'black', 'pink', 'undergarment', 'cl...",0.04,2021-11-02 19:40:32.890000+00:00,Tuesday,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,431,0,1418073537,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/wXE3J1pmXB/,1,0,NaN,2021-11-06T17:39:59.077Z,0.001229,0.002201,0.000616,0,0,0,"['grey', 'black', 'outdoor', 'aerial', 'photog...",0.06,2021-11-06 17:39:59.077000+00:00,Saturday,17
515,1,viciocassaro,ðÿ‘”,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",2.480000e+18,407,87,36,431,0,1610199243,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/CJ03kIkj8cH/,0,0,NaN,2021-11-06T17:39:59.077Z,0.008394,0.011220,0.000874,0,0,0,"['black', 'person', 'christmas', 'tree', 'clot...",0.20,2021-11-06 17:39:59.077000+00:00,Saturday,17
516,2,viciocassaro,#onestepcloser,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",1.030000e+18,407,48,36,431,0,1436953461,j_kvo3danq1yzea4j0jg.70f863db800ffee4380f1c4cd...,https://www.instagram.com/p/5JvdTIJmWU/,0,0,NaN,2021-11-06T17:39:59.077Z,0.000879,0.001615,0.000563,0,0,0,"['white', 'human', 'face', 'person', 'clothing...",0.11,2021-11-06 17:39:59.077000+00:00,Saturday,17
517,0,viciocassaro,wow.,vincenzo cassaro,"j_globe4 london, uk\nn_building6 co-founder ...",8.710000e+17,407,27,36,437,0,1418073537,j_kvo3dl7njv41lgn55.c75281bab6c50a2a8905d2be3a...,https://www.instagram.com/p/w

##Features Engineering

##Average Likes's Features

In [355]:
df['account'].value_counts().min()

1

In [356]:
df['account'].value_counts().max()

104

In [426]:
average=df.groupby(by=['account','likes'])['timestamp'].aggregate('max').to_frame('u_timestamp').reset_index()
average.iloc[:20]

,account,likes,u_timestamp
0,108harleystreet,1,2021-11-02
1,108harleystreet,2,2021-11-02
2,108harleystreet,3,2021-11-02
3,108harleystreet,4,2021-11-02
4,108harleystreet,8,2021-11-02
5,111skin,126,2021-11-02
6,111skin,139,2021-11-02
7,111skin,184,2021-11-02
8,111skin,213,2021-11-02
9,111skin,227,2021-11-02


In [412]:
average['account'].value_counts()

cheshirelashcompany      52
spacex                   20
viciocassaro             16
rozinaalimd              11
kevin                    11
                         ..
everyoungrejuvenation     1
harpalclinic              1
lmskincare                1
keviihospital             1
zelensskincare            1
Name: account, Length: 105, dtype: int64

In [413]:
Average_likes=average.groupby(by=['account'])['likes'].aggregate('mean').to_frame('Average_Likes').reset_index()

In [414]:
Average_likes

,account,Average_Likes
0,108harleystreet,3.600000
1,111skin,206.000000
2,1gemmaclare,94.000000
3,CPMedicalClinic,35.000000
4,CosmeticSurgeonsLondon,50.333333
...,...,...
100,ucl,3151.000000
101,vebleninternational,6.750000
102,viciocassaro,43.750000
103,zafnaqui,61.000000


In [415]:
Average_likes

,account,Average_Likes
0,108harleystreet,3.600000
1,111skin,206.000000
2,1gemmaclare,94.000000
3,CPMedicalClinic,35.000000
4,CosmeticSurgeonsLondon,50.333333
...,...,...
100,ucl,3151.000000
101,vebleninternational,6.750000
102,viciocassaro,43.750000
103,zafnaqui,61.000000


#Recent Likes

In [416]:
recent_likes=average.groupby(by=['account'])['likes'].aggregate('max').to_frame('u_timestamp').reset_index()
recent_likes

,account,u_timestamp
0,108harleystreet,8
1,111skin,296
2,1gemmaclare,94
3,CPMedicalClinic,35
4,CosmeticSurgeonsLondon,131
...,...,...
100,ucl,4444
101,vebleninternational,11
102,viciocassaro,87
103,zafnaqui,85


#Time feature

In [427]:
timefeature=df.groupby(by=['account','timestamp'])['likes'].aggregate('max').to_frame('time_feature_most_likes_at_timestamp').reset_index()
timefeature.iloc[:]

,account,timestamp,time_feature_most_likes_at_timestamp
0,108harleystreet,2021-11-02,8
1,111skin,2021-11-02,296
2,1gemmaclare,2021-11-02,94
3,CPMedicalClinic,2021-11-02,35
4,CosmeticSurgeonsLondon,2021-11-02,131
...,...,...,...
100,ucl,2021-11-02,4444
101,vebleninternational,2021-11-02,11
102,viciocassaro,2021-11-06,87
103,zafnaqui,2021-11-02,85


In [606]:
#day of the week user likes the most
dow=df.groupby(by=['account','likes'])['Dow'].aggregate(lambda x:stats.mode(x)[0]).to_frame('u_likes_the_most').reset_index()
dow.iloc[:]

,account,likes,u_likes_the_most
0,108harleystreet,1,Tuesday
1,108harleystreet,2,Tuesday
2,108harleystreet,3,Tuesday
3,108harleystreet,4,Tuesday
4,108harleystreet,8,Tuesday
...,...,...,...
419,viciocassaro,87,Saturday
420,zafnaqui,42,Tuesday
421,zafnaqui,56,Tuesday
422,zafnaqui,85,Tuesday


In [607]:
#Hour of the day user has likes the most
hod=df.groupby(by=['account','likes'])['HoW'].aggregate(lambda x:stats.mode(x)[0]).to_frame('hod_u_most_likes').reset_index()
hod

,account,likes,hod_u_most_likes
0,108harleystreet,1,19
1,108harleystreet,2,19
2,108harleystreet,3,19
3,108harleystreet,4,19
4,108harleystreet,8,19
...,...,...,...
419,viciocassaro,87,16
420,zafnaqui,42,20
421,zafnaqui,56,20
422,zafnaqui,85,20


#Text-related features

#Engagement factor

In [428]:
engagement_factor=df.groupby(by=['account'])['engagement_factor'].aggregate('max').to_frame('engagement_factor').reset_index()
engagement_factor.iloc[:]

,account,engagement_factor
0,108harleystreet,0.04
1,111skin,0.00
2,1gemmaclare,0.08
3,CPMedicalClinic,0.02
4,CosmeticSurgeonsLondon,0.00
...,...,...
100,ucl,0.03
101,vebleninternational,0.04
102,viciocassaro,0.20
103,zafnaqui,0.05


In [330]:
df = df.astype(str)
# df = remove_html(df['caption'][0])
df["hashtags"] = df["caption"].apply(lambda text: hashtag_extract(text))